In [1]:
!pip install numpy scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load your .npy files
def load_data(folder_landmarks, folder_motion, folder_velocity, class_label):
    X = []
    y = []
    for fname in os.listdir(folder_landmarks):
        if not fname.endswith('_landmarks.npy'):
            continue

        base = fname.replace('_landmarks.npy', '')
        try:
            l = np.load(os.path.join(folder_landmarks, fname))
            m = np.load(os.path.join(folder_motion, f"{base}_motion.npy"))
            v = np.load(os.path.join(folder_velocity, f"{base}_velocity.npy"))

            # Flatten and concatenate
            features = np.concatenate([
                l.flatten(),
                m.flatten(),
                v.flatten()
            ])
            X.append(features)
            y.append(class_label)
        except Exception as e:
            print(f"Error loading {fname}: {e}")

    return X, y

X_train, y_train = [], []
for i, cls in enumerate(['Non-suspicious', 'Suspicious']):
    Xc, yc = load_data(
        f'transformed/landmarks.npy/Train/{cls}',
        f'transformed/motion.npy/Train/{cls}',
        f'transformed/velocity.npy/Train/{cls}',
        i  # 0 or 1
    )
    X_train.extend(Xc)
    y_train.extend(yc)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train SVM
clf = SVC(kernel='rbf', C=1, gamma='scale', probability=True)
clf.fit(X_train_scaled, y_train)

# Optional: Evaluate
y_pred = clf.predict(X_train_scaled)
print(classification_report(y_train, y_pred, target_names=['Non-suspicious', 'Suspicious']))


                precision    recall  f1-score   support

Non-suspicious       0.67      0.93      0.78       116
    Suspicious       0.89      0.54      0.67       116

      accuracy                           0.74       232
     macro avg       0.78      0.74      0.73       232
  weighted avg       0.78      0.74      0.73       232

